In [1]:
import pandas as pd
import numpy as np
import numpy_financial as npf


################################################################################
#Functions

### Prompt user to input a percentage (with error-checking)
def ask_pct(pct, question):
    while True:
        try:
            pct = float(input(question))
        except ValueError:
            print("Value must be given as a number between 0 and 100.")
            continue
        
        if pct == 0:
            break

        if 1 < pct <= float(100):
            pct = pct/100
            break
        
        if 0 < pct <= 1:
            confirm = input("Value should be given as a percent from 0-100 and not as a proportion from 0-1. Are you sure you would like to continue (y/n)?")
            if confirm.lower() == 'y' or confirm.lower() == 'yes':
                pct = pct/100
                break
            if confirm.lower() == 'n' or confirm.lower() == 'no':
                continue
            else:
                print('I did not understand that. Type "y" for yes or "n" for no.')
        else:
            print("Value must be given as a number between 0 and 100.")
            continue
    return pct

### Prompt user to input a number (with error-checking
def ask_num(num, question):
    while True:
        try:
            num = float(input(question))
        except ValueError:
            print("Value must be given as a positive number.")
            continue

        if 1 < num:
            break
        
        if 0 < num <= 1:
            confirm = input("Value should be given as an absolute number and not as a proportion from 0-1. Are you sure you would like to continue (y/n)?")
            if confirm.lower() == 'y' or confirm.lower() == 'yes':
                break
            if confirm.lower() == 'n' or confirm.lower() == 'no':
                continue
            else:
                print('I did not understand that. Type "y" for yes or "n" for no.')
        else:
            print("Value must be given as a positive number.")
            continue
    return num

### Prompt user to input a number with default = 1000 (with error-checking)
def ask_par(num, question):
    while True:
        num = input(question)
        
        if num == "":
            num = float(1000)
            break
            
        try:
            num = float(num)
        except ValueError:
            print("Value must be given as a positive number.")
            continue

        if 1 < num:
            break
        
        if 0 < num <= 1:
            confirm = input("Value should be given as an absolute number and not as a proportion from 0-1. Are you sure you would like to continue (y/n)?")
            if confirm.lower() == 'y' or confirm.lower() == 'yes':
                break
            if confirm.lower() == 'n' or confirm.lower() == 'no':
                continue
            else:
                print('I did not understand that. Type "y" for yes or "n" for no.')
        else:
            print("Value must be given as a positive number.")
            continue
    return num


################################################################################
#Program

### Introduction
print('Welcome to the WACC Estimator!')
name = str(input('Please enter your name: '))
print(f'Hello {name}! Lets begin.\n')


### Initialize essential variables
e = 0
d = 0
ps = 0
ke = 0
kd = 0
kp = 0


### Capital Structure
while True:
    ps = ask_pct(ps, "What percentage of the company is composed of preferred stock? (0-100)")
    e = ask_pct(e, "What percentage of the company is composed of common stock? (0-100)")
    d = ask_pct(d, "What percentage of the company is composed of debt? (0-100)")
    if e + d + ps - 1 < 0.000001: #floats throw this off just a bit; try 60/30/10
        break
    print("\nThe company's capital structure must total to 100%.")

if ps > 0.2:
    print("\nSo many preferred shares! Are you a founder? Or did you just fall from heaven ...you little angel, you.\n")
elif d > 0.5:
    print("\nStranger, strangerr. Now that's some leverage!\n")
elif e > 0.6:
    print("\nWhat's up with all the equity? Do you have little to no ~interest~ in a loan? ;)\n")
else:
    print("\nNow that's a balanced capital structure!\n")

    
# Initialize equity values
pe0 = 0
de1 = 0
g = 0

# Calculate cost of equity
if e > 0:
    print('\nNow, I need some inputs to find the cost of equity!\n')
    pe0 = ask_num(pe0,"What is the company's current common stock price?")
    de1 = ask_num(de1, "What is the company's next (annual) dividend payout?")
    g = ask_pct(g,"What is the company's anticipated growth rate? (0-100)")
    ke = de1/pe0+g
    print('\nCost of Equity: STORED!')
else:
    print('\nYou have zero equity, therefore your cost of equity is zero!\n')

# Initialize debt values
term = 0
cr = 0
f = 0
par = 1000
pd0 = 0
t = 0

# Calculate cost of debt
if d > 0:
    print('\nMoving on to cost of debt!\n')
    term = ask_num(term, "What is the company's term of debt in years?")
    cr = ask_pct(cr, "What is the coupon rate on debt as an APR (0-100)?")
    f = ask_num(f, "What is the frequency of coupon payments per year?")
    par = ask_par(par, "What is the par value of debt (Default Value = $1000. Press Enter to accept default.)?")
    pd0 = ask_num(pd0, "What is the price of debt as a percentage of par (eg. 85.12, 94, 108)?")
    t = ask_pct(t, "What is the company's tax rate as a percentage? (0-100)")
    
    pmts = [-pd0]
    for per in range(int(term*f)):
        pmts.append(cr*100/f)
    pmts[-1] += 100
    kd = f*npf.irr(pmts)
    print('\nCost of Debt: STORED! Tax Shield: STORED!')
else:
    print('\nYou have no debt, therefore your cost of debt is zero!\n')

# Initialize preferred stock values
pp0 = 0
dp1 = 0

# Calculate cost of preferred stock
if ps > 0:
    pp0 = ask_num(pp0, "What is the company's preferred stock price?")
    dp1 = ask_num(dp1, "What is the company's next (annual) preferred dividend payout?")
    kp = dp1/pp0
    print('\nCost of Preferred Stock: STORED!')
else:
    print('\nYou have no preferred stock, therefore your cost of preferred stock is zero!\n')
    
# Printing a thank you and add some suspense!!!
print('\nThanks for all your input.')
while True:
    finale = str(input('Enter \'wahoo\' to see your calculated WACC value!'))
    if finale.lower() == 'wahoo':
        break;
    else: 
        print("Come on, dont be a party pooper! Enter wahoo!")  

# Calculate WACC and report
wacc = e*ke + d*kd*(1-t) + ps*kp

vals = {'Capital Structure':[ps*100, e*100, d*100, 100], 'Cost of Capital':[kp*100, ke*100, kd*100, wacc*100]}
df = pd.DataFrame(data = vals, index = [ 'Preferred Stock', 'Equity', 'Debt (t='+str(t*100)+'%)', 'WACC'])

print("\n Your company's WACC report: \n")
print(df.round(2))
print('\nYour estimated WACC value is: ' + str(round(wacc*100,2)) + '%')

Welcome to the WACC Estimator!
Hello Rukh! Lets begin.


So many preferred shares! Are you a founder? Or did you just fall from heaven ...you little angel, you.


Now, I need some inputs to find the cost of equity!


Cost of Equity: STORED!

Moving on to cost of debt!

I did not understand that. Type "y" for yes or "n" for no.

Cost of Debt: STORED! Tax Shield: STORED!

Cost of Preferred Stock: STORED!

Thanks for all your input.

 Your company's WACC report: 

                 Capital Structure  Cost of Capital
Preferred Stock               55.0            21.43
Equity                        20.0             5.80
Debt (t=35.0%)                20.0             6.63
WACC                         100.0            13.81

Your estimated WACC value is: 13.81%
